In [ ]:
import csv
with open('DocMeta2.tsv') as f:
    data=f.readlines()

In [ ]:
import nltk 
from nltk.tokenize import word_tokenize,sent_tokenize
news={}
category={'none':0}
subcategory={'none':0}
for i in data:
    tp=i.strip().split('\t')
    if len(tp)<5:
        continue
    news[tp[1]]=[tp[2],tp[3],word_tokenize(tp[6].lower())]
    if tp[2] not in category:
        category[tp[2]]=len(category)
    if tp[3] not in subcategory:
        subcategory[tp[3]]=len(subcategory)

In [ ]:
with open('ClickData2.tsv')as f:
    user=f.readlines()
userid_dict={}
for i in user:
    tr=i.strip().split('\t')
    userid=tr[0]
    if userid not in  userid_dict:
        userid_dict[userid]=len(userid_dict)

In [ ]:
with open('userid.txt','r')as f:
    rawuserlabel=[x.strip().split('\t') for x in f.readlines()]

In [ ]:
userlabeldict={}
genderdict={'M':0,'F':1}

for i in rawuserlabel:
    if len(i)==2:
        userlabeldict[i[0]]=[0,0]
    else:
        userlabeldict[i[0]]=[genderdict[i[2]],1]


In [ ]:
for i in userid_dict:
    if i not in userlabeldict:
        userlabeldict[i]=[0,0]

In [ ]:


word_dict0={'PADDING':[0,999999]}

for i in news:
    for j in news[i][2]:
        if j in word_dict0:
            word_dict0[j][1]+=1
        else:
            word_dict0[j]=[len(word_dict0),1]


In [ ]:


word_dict={}
for i in word_dict0:
    if word_dict0[i][1]>=1:
        word_dict[i]=[len(word_dict),word_dict0[i][1]]
print(len(word_dict),len(word_dict0))


In [ ]:
import numpy as np
embdict={}
plo=0
import pickle
with open('/data/wuch/glove.840B.300d.txt','rb')as f:
    linenb=0
    while True:
        j=f.readline()
        if len(j)==0:
            break
        k = j.split()
        word=k[0].decode()
        linenb+=1
        if len(word) != 0:
            tp=[float(x) for x in k[1:]]
            if word in word_dict:
                embdict[word]=tp
                if plo%100==0:
                    print(plo,linenb,word)
                plo+=1



In [ ]:
from numpy.linalg import cholesky
word_dict1=word_dict
print(len(embdict),len(word_dict1))
print(len(word_dict1))
lister=[0]*len(word_dict1)
xp=np.zeros(300,dtype='float32')

cand=[]
for i in embdict.keys():
    lister[word_dict1[i][0]]=np.array(embdict[i],dtype='float32')
    cand.append(lister[word_dict1[i][0]])
cand=np.array(cand,dtype='float32')

mu=np.mean(cand, axis=0)
Sigma=np.cov(cand.T)

norm=np.random.multivariate_normal(mu, Sigma, 1)
print(mu.shape,Sigma.shape,norm.shape)

for i in range(len(lister)):
    if type(lister[i])==int:
        lister[i]=np.reshape(norm, 300)
lister[0]=np.zeros(300,dtype='float32')
lister=np.array(lister,dtype='float32')
print(lister.shape)



In [ ]:
news_words=[[0]*30]
news_index={'0':0}
for i in news:
    tp=[]
    news_index[i]=len(news_index)
    for j in news[i][2]:
        if j in word_dict:
            tp.append(word_dict[j][0])
    tp=tp[:30]
    news_words.append(tp+[0]*(30-len(tp)))

In [ ]:
import itertools
import random
import random
def newsample(neglist,ratio):
    if ratio >len(neglist):
        return random.sample(neglist*(ratio//len(neglist)+1),ratio)
    else:
        return random.sample(neglist,ratio)
npratio=4
all_train_id=[]
all_train_pn=[]    
all_labeler=[]

all_test_id=[]
all_test_pn=[]    
all_test_labeler=[]
all_test_index=[]

all_user_pos=[]
all_test_user_pos=[]

for line in user:
    linesplit=line.strip().split('\t')
    userid=linesplit[0]

    impre=[x.split('#TAB#') for x in linesplit[2].split('#N#')]

    trainpos=[x[0].split() for x in impre]
    trainneg=[x[1].split() for x in impre]
     
    poslist=list(itertools.chain(*(trainpos)))
    neglist=list(itertools.chain(*(trainneg)))
    #print(poslist)
    
    
    if len(linesplit)==4:
        impre=[x.split('#TAB#') for x in linesplit[3].split('#N#')]
        testpos=[x[0].split() for x in impre]
        testneg=[x[1].split() for x in impre]
        
        
        for i in range(len(testpos)):
            sample_index=[]
            sample_index.append(len(all_test_pn))
            posset=list(set(poslist))
            allpos=[news_index[p] for p in random.sample(posset,min(50,len(posset)))[:50]]
            allpos+=[0]*(50-len(allpos))
    
            
            for j in testpos[i]:
                all_test_pn.append(news_index[j])
                all_test_labeler.append(1)
                all_test_id.append(userid_dict[userid])
                all_test_user_pos.append(allpos)
                
            for j in testneg[i]:
                all_test_pn.append(news_index[j])
                all_test_labeler.append(0)
                all_test_id.append(userid_dict[userid])
                all_test_user_pos.append(allpos)
            sample_index.append(len(all_test_pn))
            all_test_index.append(sample_index)
            
    #neglistnn=list(set(neglist)-set(poslist))

            
    for impre_id in range(len(trainpos)):
        for pos_sample in trainpos[impre_pos_id]:
            #tql=list(set(neglist)-set(trainpos[mp]))
            all_sample=newsample(trainneg[impre_id],npratio)
            all_sample.append(pos_sample)
            templabel=[0]*npratio+[1]
            labelid=list(range(npratio+1))
            random.shuffle(labelid)

            
            sample=[]
            label=[]
            for j in labelid:
                sample.append(news_index[all_sample[j]])
                label.append(templabel[j])
            
            posset=list(set(poslist)-set([pos_sample]))
            allpos=[news_index[p] for p in random.sample(posset,min(50,len(posset)))[:50]]
            allpos+=[0]*(50-len(allpos))
            all_train_pn.append(yp)
            all_labeler.append(yplb)
            all_train_id.append(userid_dict[userid])
            all_user_pos.append(allpos)


In [ ]:
inverseuserid_dic={userid_dict[x]:x for x in userid_dict}

In [ ]:
import numpy as np
news_words=np.array(news_words,dtype='int32') 

all_train_pn=np.array(all_train_pn,dtype='int32')
all_labeler=np.array(all_labeler,dtype='int32')
all_train_id=np.array(all_train_id,dtype='int32')
all_test_pn=np.array(all_test_pn,dtype='int32')
all_test_labeler=np.array(all_test_labeler,dtype='int32')
all_test_id=np.array(all_test_id,dtype='int32')
all_user_pos=np.array(all_user_pos,dtype='int32')
all_test_user_pos=np.array(all_test_user_pos,dtype='int32')

In [ ]:
all_train_gender_label=np.array([userlabeldict[inverseuserid_dic[x]] for x in all_train_id],dtype='int32') 
all_test_gender_label=np.array([userlabeldict[inverseuserid_dic[x]] for x in all_test_id],dtype='int32') 

In [ ]:

npratio=4   
user_label_gender=[]
user_pos=[]

for line in user:
    linesplit=line.strip().split('\t')
    userid=linesplit[0]

    impre=[x.split('#TAB#') for x in linesplit[2].split('#N#')]

    trainpos=[x[0].split() for x in impre]
     
    poslist=list(itertools.chain(*(trainpos)))
    if userid not in userlabeldict or userlabeldict[userid][1]==0:
        continue
     
            
    allpos=[news_index[p] for p in random.sample(poslist,min(50,len(poslist)))[:50]]
    allpos+=[0]*(50-len(allpos))

    user_label_gender.append(userlabeldict[userid][0])
    user_pos.append(allpos)


In [ ]:
user_pos_words=news_words[np.array(user_pos,dtype='int32')]
user_label_gender=np.array(user_label_gender,dtype='float32')

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from keras.utils.np_utils import to_categorical

from keras.layers import *
from keras.models import Model, load_model
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers #keras2
from keras.utils import plot_model
from sklearn.metrics import accuracy_score, classification_report
from keras.optimizers import *
import keras

In [ ]:
def generate_batch_data_random(all_train_pn,all_labeler,all_train_id,batch_size):
    idx = np.arange(len(all_labeler))
    np.random.shuffle(idx)
    y=all_labeler
    batches = [idx[range(batch_size*i, min(len(y), batch_size*(i+1)))] for i in range(len(y)//batch_size+1)]

    while (True):
        for i in batches:
            itx = news_words[all_train_pn[i]]
            usx=news_words[all_user_pos[i]]
            yy=all_labeler[i]
            
            yield ([itx,usx], [yy])

In [ ]:
def generate_batch_data_random_gender(all_train_pn,all_labeler,all_train_id,batch_size):
    idx = np.arange(len(all_labeler))
    np.random.shuffle(idx)
    y=all_labeler
    batches = [idx[range(batch_size*i, min(len(y), batch_size*(i+1)))] for i in range(len(y)//batch_size+1)]

    while (True):
        for i in batches:
            itx = news_words[all_train_pn[i]]
            usx=news_words[all_user_pos[i]]
            yy=all_labeler[i]
            yg=all_train_gender_label[:,0][i]
            ygmask=all_train_gender_label[:,1][i]
            
            yield ([itx,usx], [yy,yg,np.zeros((batch_size,),dtype='float32')],[np.ones((batch_size,),dtype='float32'),ygmask,np.ones((batch_size,),dtype='float32')])

In [ ]:
def generate_batch_data(all_train_pn,all_labeler,all_train_id,batch_size):
    idx = np.arange(len(all_labeler))
    y=all_labeler
    batches = [idx[range(batch_size*i, min(len(y), batch_size*(i+1)))] for i in range(len(y)//batch_size+1)]

    while (True):
        for i in batches:
            itx = news_words[all_train_pn[i]]

            usx=news_words[all_test_user_pos[i]]
            yy=all_labeler[i]
            yield ([itx,usx], [yy])

In [ ]:

def dcg_score(y_true, y_score, k=10):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)


def ndcg_score(y_true, y_score, k=10):
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_score, k)
    return actual / best


def mrr_score(y_true, y_score):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order)
    rr_score = y_true / (np.arange(len(y_true)) + 1)
    return np.sum(rr_score) / np.sum(y_true)


In [ ]:

from keras.layers import *


class Attention(Layer):

    def __init__(self, nb_head, size_per_head, **kwargs):
        self.nb_head = nb_head
        self.size_per_head = size_per_head
        self.output_dim = nb_head*size_per_head
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.WQ = self.add_weight(name='WQ', 
                                  shape=(input_shape[0][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WK = self.add_weight(name='WK', 
                                  shape=(input_shape[1][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WV = self.add_weight(name='WV', 
                                  shape=(input_shape[2][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        super(Attention, self).build(input_shape)
        
    def Mask(self, inputs, seq_len, mode='mul'):
        if seq_len == None:
            return inputs
        else:
            mask = K.one_hot(seq_len[:,0], K.shape(inputs)[1])
            mask = 1 - K.cumsum(mask, 1)
            for _ in range(len(inputs.shape)-2):
                mask = K.expand_dims(mask, 2)
            if mode == 'mul':
                return inputs * mask
            if mode == 'add':
                return inputs - (1 - mask) * 1e12
                
    def call(self, x):
        #如果只传入Q_seq,K_seq,V_seq，那么就不做Mask
        #如果同时传入Q_seq,K_seq,V_seq,Q_len,V_len，那么对多余部分做Mask
        if len(x) == 3:
            Q_seq,K_seq,V_seq = x
            Q_len,V_len = None,None
        elif len(x) == 5:
            Q_seq,K_seq,V_seq,Q_len,V_len = x
        #对Q、K、V做线性变换
        Q_seq = K.dot(Q_seq, self.WQ)
        Q_seq = K.reshape(Q_seq, (-1, K.shape(Q_seq)[1], self.nb_head, self.size_per_head))
        Q_seq = K.permute_dimensions(Q_seq, (0,2,1,3))
        K_seq = K.dot(K_seq, self.WK)
        K_seq = K.reshape(K_seq, (-1, K.shape(K_seq)[1], self.nb_head, self.size_per_head))
        K_seq = K.permute_dimensions(K_seq, (0,2,1,3))
        V_seq = K.dot(V_seq, self.WV)
        V_seq = K.reshape(V_seq, (-1, K.shape(V_seq)[1], self.nb_head, self.size_per_head))
        V_seq = K.permute_dimensions(V_seq, (0,2,1,3))
        #计算内积，然后mask，然后softmax
        A = K.batch_dot(Q_seq, K_seq, axes=[3,3]) / self.size_per_head**0.5
        A = K.permute_dimensions(A, (0,3,2,1))
        A = self.Mask(A, V_len, 'add')
        A = K.permute_dimensions(A, (0,3,2,1))    
        A = K.softmax(A)
        #输出并mask
        O_seq = K.batch_dot(A, V_seq, axes=[3,2])
        O_seq = K.permute_dimensions(O_seq, (0,2,1,3))
        O_seq = K.reshape(O_seq, (-1, K.shape(O_seq)[1], self.output_dim))
        O_seq = self.Mask(O_seq, Q_len, 'mul')
        return O_seq
        
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], self.output_dim)

In [ ]:
import itertools
import random

from keras import objectives
results=[]
 
keras.backend.clear_session()

MAX_SENT_LENGTH=30
MAX_SENTS=50


sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')

embedding_layer = Embedding(len(word_dict), 300, weights=[lister],trainable=True)

embedded_sequences = embedding_layer(sentence_input)
embedded_sequences=Dropout(0.2)(embedded_sequences)

#bias-aware

word_rep =Attention(16,16)([embedded_sequences,embedded_sequences,embedded_sequences])
word_rep=Dropout(0.2)(word_rep)

attention_word = Dense(200,activation='tanh')(word_rep)
attention_word = Flatten()(Dense(1)(attention_word))
attention_word = Activation('softmax')(attention_word)
news_rep=keras.layers.Dot((1, 1))([word_rep, attention_word])



news_encoder = Model([sentence_input], news_rep)


history_input = keras.Input((MAX_SENTS,MAX_SENT_LENGTH,), dtype='int32') 
his_vecs =TimeDistributed(news_encoder)(history_input)

his_vecs=Attention(16,16)([his_vecs,his_vecs,his_vecs])
attention_news = Dense(200,activation='tanh')(his_vecs)
attention_news =  Flatten()(Dense(1)(attention_news))
attention_news = Activation('softmax')(attention_news)
user_emb=keras.layers.Dot((1, 1))([his_vecs, attention_news])

#bias-free

his_vecs2=Attention(16,16)([his_vecs,his_vecs,his_vecs])
attention_news2 = Dense(200,activation='tanh')(his_vecs2)
attention_news2 =  Flatten()(Dense(1)(attention_news2))
attention_news2 = Activation('softmax')(attention_news2)
user_emb2=keras.layers.Dot((1, 1))([his_vecs2, attention_news2])



candidates = keras.Input((1+npratio,MAX_SENT_LENGTH,), dtype='int32') 
candidate_vecs = TimeDistributed(news_encoder)(candidates)

user_emb_unified=add([user_emb,user_emb2])

logits = keras.layers.dot([user_emb_unified, candidate_vecs], axes=-1)
logits = keras.layers.Activation(keras.activations.softmax)(logits)




loss_orth=Lambda(lambda x:K.batch_dot(x[0],x[1],axes=-1)/K.sqrt(K.sum(K.square(x[0]),axis=1)*K.sum(K.square(x[1]),axis=1)))([user_emb,user_emb2])
def orthloss(y_true, y_pred):
    return K.mean(K.abs(loss_orth))

outattribute=Dense(1,activation='sigmoid')(Dense(128)(l_att2))

model = Model([candidates,history_input], [logits,outattribute,loss_orth])
model.trainable=True

candidate_one = keras.Input((MAX_SENT_LENGTH,))
candidate_one_vec = news_encoder([candidate_one])

score = keras.layers.Activation(keras.activations.sigmoid)(
    keras.layers.dot([user_emb2, candidate_one_vec], axes=-1))

model_bias_free = keras.Model([candidate_one,history_input], score)



model.compile(loss=['categorical_crossentropy','binary_crossentropy',orthloss],loss_weights=[1.,0.5,0.5], 
              optimizer=Adam(lr=0.0001), metrics=['acc'])



usermodel = Model([candidates,history_input], [user_emb2])
 
userv=Input((256,),dtype='float32')
out=Dense(1,activation='sigmoid')(Dense(256)(userv))

usergenderdis = Model([userv], [out])

gan_output = usergenderdis(usermodel([candidates,history_input]))

gan = keras.models.Model([candidates,history_input], [logits,gan_output,loss_orth])
gan.compile(optimizer=Adam(lr=0.0001), metrics=['acc'],loss=['categorical_crossentropy','binary_crossentropy',orthloss],loss_weights=[1.,0.5,0.5])
usergenderdis.compile(loss=['binary_crossentropy'], optimizer=Adam(lr=0.0001), metrics=['acc'])
for epoch in range(1):
    traingen=generate_batch_data_random_gender(all_train_pn,all_labeler,all_train_id, 30)
    iterations=len(all_train_id)//30
    step=0
    historicala=[]
    historicald=[]
    for data in traingen:
        disvec=usermodel.predict(data[0])
        usergenderdis.trainable = True
        d_loss =usergenderdis.train_on_batch(disvec,[1-data[1][1]],sample_weight=data[2][1])
        usergenderdis.trainable = False
        a_loss =gan.train_on_batch(data[0],[data[1][0],data[1][1],data[1][2]],sample_weight=data[2])
        step+=1
        historicald.append(d_loss)
        historicala.append(a_loss)
        historicald=historicald[-10:]
        historicala=historicala[-10:]
        if step%10==0:
            print(d_loss)
            print(np.mean(historicald,axis=0))
            print(a_loss)
            print(np.mean(historicala,axis=0))
        if step==iterations:
            break
    valgen=generate_batch_data(all_test_pn,all_test_labeler,all_test_id, 94)
    cr = model_bias_free.predict_generator(valgen, steps=len(all_test_id)//94,verbose=1)
    from sklearn.metrics import roc_auc_score
    all_auc=[]
    all_mrr=[]
    all_ndcg=[]
    all_ndcg2=[]
    for m in all_test_index:
        if np.sum(all_test_labeler[m[0]:m[1]])!=0 and m[1]<len(cr):
    
            all_auc.append(roc_auc_score(all_test_labeler[m[0]:m[1]],cr[m[0]:m[1],0]))
            all_mrr.append(mrr_score(all_test_labeler[m[0]:m[1]],cr[m[0]:m[1],0]))
            all_ndcg.append(ndcg_score(all_test_labeler[m[0]:m[1]],cr[m[0]:m[1],0],k=5))
            all_ndcg2.append(ndcg_score(all_test_labeler[m[0]:m[1]],cr[m[0]:m[1],0],k=10))
    results.append([np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg2)])
    print(np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg2))

In [ ]:
userindex=np.array(random.sample(np.where(user_label_gender==0)[0].tolist(),len(np.where(user_label_gender==1)[0]))+np.where(user_label_gender==1)[0].tolist())

In [ ]:
modeluser = Model([history_input], [user_emb])

useremb=modeluser.predict(user_pos_words[userindex])

In [ ]:
modeluser2 = Model([history_input], [user_emb2])

useremb2=modeluser2.predict(user_pos_words[userindex])

In [ ]:
newsemb=news_encoder.predict(news_words)

In [ ]:
gender_index=np.arange(len(useremb))
np.random.shuffle(gender_index)

In [ ]:
k=10
topnews=news_words[np.dot(useremb2,np.transpose(newsemb)).argsort(axis=1)[:, -k:][:, ::-1]]
toplabel=user_label_gender[userindex]

model.trainable=False

topK_input = keras.Input((k,MAX_SENT_LENGTH,), dtype='int32') 
topKrep=TimeDistributed(news_encoder)(topK_input)

topK_score_news=topKrep#concatenate([topK_score_dense,topKrep])


attention = Dense(200,activation='tanh')(topKrep)
attention = Flatten()(Dense(1)(attention))
attention = Activation('softmax')(attention)
user_gen_rep=keras.layers.Dot((1, 1))([topKrep, attention])
out=Dense(1,activation='sigmoid')(user_gen_rep)
usergendermodel = Model([topK_input], [out])
usergendermodel.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['acc'])
for _ in range(10):
    usergendermodel.fit([topnews[gender_index[:int(0.8*len(gender_index))]]],toplabel[gender_index[:int(0.8*len(gender_index))]],batch_size=64)
    pred=usergendermodel.predict([topnews[gender_index[int(0.8*len(gender_index)):]]],batch_size=32)
    print(classification_report(toplabel[gender_index[int(0.8*len(gender_index)):]],np.round(pred),digits=4))
    print(accuracy_score(toplabel[gender_index[int(0.8*len(gender_index)):]],np.round(pred)))